# 13-validate-csv
> Validating a csv file

This notebook validates the format of csv files.

# Helpful packages

In [ ]:
#all_no_test

In [ ]:
#export text_preprocessing
import pandas as pd
import re
import os.path

# Set the file path

You can change 'base_prefix' variable below according to your computer environment. In this example, Soyeon's local file path was used.

In [ ]:
#base_prefix = os.path.expanduser('~/Box Sync/DSI Documents/')
base_prefix = '/data/p_dsi/wise/data/'
file_directory = base_prefix + 'cleaned_data/csv_files/megadata.csv'
df = pd.read_csv(file_directory)

In [ ]:
df.head()

,id,transcript_filepath,wave_filepath,speech,start_timestamp,end_timestamp,label,transcriber_id
0,129-1,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,can you do it really quickly.,00:00:00.00,00:02:02.12,NEU,198
1,129-1,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,just stick it in your cubby really quick pleas...,00:00:00.00,00:02:02.12,NEU,198
2,129-1,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,thank you.,00:00:00.00,00:02:02.12,NEU,198
3,129-1,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,(okay) so we are going to >,00:00:00.00,00:02:02.12,NEU,198
4,129-1,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,you're gonna do the same thing name.,00:00:00.00,00:02:02.12,NEU,198


# Function #1: Checking the labels
This function checks if there is any labels with typos. If used label is default('PRS', 'REP', 'OTR', 'NEU'), keep the value of the variable 'accepted_labels" as None. If you want to change the label, then assign a list of new labels to the 'accepted_labels' variable.

In [ ]:
#export text_preprocessing
def check_label(dataframe, accepted_labels = None):
    """
    Validate labels
    
    Parameters
    ----------
    dataframe : dataframe
        a dataframe that you want to check labels
    accepted_labels: either None or a list of new labels
        None: When you want to use the default labels ('PRS', 'REP', 'OTR', 'NEU')
        a list of new labels: When you want to change the labels (ex. ['OTR_academic, 'OTR_behavior])
        
    Returns
    -------
    dataframe
        rows of which label is not an element of the list we input
    """
    if accepted_labels == None:
        label_list = ['PRS', 'REP', 'OTR', 'NEU']
        return dataframe[~dataframe['label'].isin(label_list)]
    else:
        label_list = accepted_labels
        return dataframe[~dataframe['label'].isin(label_list)]

## Test 1
The result below shows a row of which label is typo. The label is "NUE". This should be corrected to "NEU".

In [ ]:
check_label(df)

,id,transcript_filepath,wave_filepath,speech,start_timestamp,end_timestamp,label,transcriber_id
6910,255-2,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,okay.,00:00:00.00,00:02:06.25,NUE,198


## Test 2
This time. accepted_labels is not None. I assigned a new list of labels. In this case, we are accepting only 'NEU', 'PRS', and 'OTR'. Therefore, the rows of which 'label' is "REP" are returned saying the labels are wrong.

In [ ]:
check_label(df, accepted_labels = ['NEU', 'PRS', 'OTR']).head()

,id,transcript_filepath,wave_filepath,speech,start_timestamp,end_timestamp,label,transcriber_id
22,129-1,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,we need you over here.,00:00:00.00,00:02:02.12,REP,198
23,129-1,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,come on quickly.,00:00:00.00,00:02:02.12,REP,198
35,129-1,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,(okay now) remember we were working on just no...,00:00:00.00,00:02:02.12,REP,198
40,129-1,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,so don't ask me again now where do we go today...,00:00:00.00,00:02:02.12,REP,198
44,129-1,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,thank you for raising your hand.,00:00:00.00,00:02:02.12,REP,198


## Function #2: Checking the format of start/end_timestamp

This function validates `start_timestamp` and `end_timestamp`. There are some timestamp in incorrect format such as "00:00:04 .44", and "00:04:53:33". It finds which timestamp is in wrong format, and ask a user if he/she wants to change it. If the user wants to change it, he/she can put correct timestamp and save it.

In [ ]:
#export text_preprocessing
def check_ts(dataframe, outfile_name = 'megadata_final', interactive=False):
    """
    Find start/end_timestamp in wrong format
    If an user wants to change the wrong start_timestamp and end_timestamp, he/she can change it and save it.
    
    Parameters
    ----------
    dataframe: dataframe
        A dataframe you want to validate in terms of start_timestamp and end_timestamp
    outfile_name: string (default 'megadata_final')
        String of the filename you want the new file saved to
    interactive: boolean (default False)
        Determines whether you want to interactively change the values or just see the errors.
        
    Returns
    -------
    dataframe
        Rows having start/end_timestamp in wrong format
        It also asks users if he/she wants to change the wrong timestamp.
        If the user wants to change it, he/she can type a correct timestamp and save it.
    """
    # Find the rows having start/end_timestamp in wrong format
    df_ts_typo = dataframe[(~dataframe.start_timestamp.str.match('\d\d:\d\d:\d\d\.\d\d')) | (~dataframe.end_timestamp.str.match('\d\d:\d\d:\d\d\.\d\d'))]
    display(df_ts_typo)
    
    # Get the list of timestamp in wrong format
    df_start_ts_typo = dataframe[(~dataframe.start_timestamp.str.match('\d\d:\d\d:\d\d\.\d\d'))] 
    df_end_ts_typo = dataframe[(~dataframe.end_timestamp.str.match('\d\d:\d\d:\d\d\.\d\d'))]
    total_typos = len(df_start_ts_typo) + len(df_end_ts_typo)
    
    # Get unique timestamps
    start_ts_list = df_start_ts_typo['start_timestamp'].unique().tolist()
    end_ts_list = df_end_ts_typo['end_timestamp'].unique().tolist()
    combine = start_ts_list + end_ts_list
    combine = list(set(combine))
    
    # Show the number of timestamp in wrong format
    print('There are', len(combine), 'unique timestamps in wrong format with a total of', total_typos, 'incorrect rows.')
    
    if interactive:
    
        # Check all the timestamp in incorrect format
        for i in combine:
            user_answer1 = input("Do you want to change {}? y/n \n".format(i))
            if user_answer1 == "y":
                user_answer2 = input("What is the correct value?\n")
                dataframe.loc[dataframe.start_timestamp == i, 'start_timestamp'] = user_answer2
                dataframe.loc[dataframe.end_timestamp == i, 'end_timestamp'] = user_answer2
                print("The new value has been updated.")
            else:
                print("Please go to the original docx file and correct it.")

        # Ask a user if he/she wants to save the change
        user_answer3 = input("Do you want to save the changes? y/n \n")
        if user_answer3 == "y":
            output_filepath = base_prefix + 'cleaned_data/csv_files/' + outfile_name + '.csv'
            dataframe.to_csv(output_filepath, index=False)
            print("You just saved your updated changes. The updated file is: ", output_filepath, '.')
        else:
            print("You did not save the change.")
    
    return

## Test 1
There are 108 rows in megadata dataframe having start/end_timestamp in wrong format, and their unique values are "00:07:45:15", and "00:04:32-00". The first one is incorrect because the unit for milisecond should be a dot(.), not a colon(:). The latter one is having hypen(-) between the digits. After the function shows all rows having incorrect start/end_timestamp, it summurizes the result. It also asks to a user if he/she wants to change the timestamp. If the user types 'y', the function asks what the correct value is. Finally, the user can decide if he/she wants to save the change.

In [ ]:
check_ts(df)

,id,transcript_filepath,wave_filepath,speech,start_timestamp,end_timestamp,label,transcriber_id
2562,129-3,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,what about these two?,00:02:01.13,00:04:32-00,NEU,198
2563,129-3,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,yeah.,00:02:01.13,00:04:32-00,NEU,198
2564,129-3,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,yeah.,00:02:01.13,00:04:32-00,NEU,198
2565,129-3,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,(well) yeah.,00:02:01.13,00:04:32-00,NEU,198
2566,129-3,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,I guess.,00:02:01.13,00:04:32-00,NEU,198
...,...,...,...,...,...,...,...,...
6215,053-2,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,(good) good job.,00:07:45:15,00:09:17.22,PRS,198
6216,053-2,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,(okay) let's look at another one.,00:07:45:15,00:09:17.22,OTR,198
6217,053-2,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,part two.,00:07:45:15,00:09:17.22,OTR,198
6218,053-2,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,ready?,00:07:45:15,00:09:17.22,OTR,198


There are 2 unique timestamps in wrong format with a total of 108 incorrect rows.


## Test 2

After using the validation function, we can tell that there is no more timestamp in wrong format.

In [ ]:
check_ts(df, outfile_name = 'megadata_final', interactive=True)

,id,transcript_filepath,wave_filepath,speech,start_timestamp,end_timestamp,label,transcriber_id
2562,129-3,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,what about these two?,00:02:01.13,00:04:32-00,NEU,198
2563,129-3,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,yeah.,00:02:01.13,00:04:32-00,NEU,198
2564,129-3,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,yeah.,00:02:01.13,00:04:32-00,NEU,198
2565,129-3,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,(well) yeah.,00:02:01.13,00:04:32-00,NEU,198
2566,129-3,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,I guess.,00:02:01.13,00:04:32-00,NEU,198
...,...,...,...,...,...,...,...,...
6215,053-2,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,(good) good job.,00:07:45:15,00:09:17.22,PRS,198
6216,053-2,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,(okay) let's look at another one.,00:07:45:15,00:09:17.22,OTR,198
6217,053-2,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,part two.,00:07:45:15,00:09:17.22,OTR,198
6218,053-2,/data/p_dsi/wise/data/cleaned_data/cleaned_tra...,/data/p_dsi/wise/data/Audio Files & Transcript...,ready?,00:07:45:15,00:09:17.22,OTR,198


There are 2 unique timestamps in wrong format with a total of 108 incorrect rows.
Do you want to change 00:04:32-00? y/n 
y
What is the correct value?
00:04:32.00
The new value has been updated.
Do you want to change 00:07:45:15? y/n 
y
What is the correct value?
00:07:45.15
The new value has been updated.
Do you want to save the changes? y/n 
y
You just saved your updated changes. The updated file is:  /data/p_dsi/wise/data/cleaned_data/csv_files/megadata_final.csv .


In [ ]:
check_ts(df)

,id,transcript_filepath,wave_filepath,speech,start_timestamp,end_timestamp,label,transcriber_id


There are 0 unique timestamps in wrong format with a total of 0 incorrect rows.
